<a href="https://colab.research.google.com/github/20023170uhi/20023170_DataAnalytics/blob/main/20023170_assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

# Methodology

In [ ]:
import pandas as pd
import numpy as np

%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

import shutil 

df1 = pd.read_csv('https://raw.githubusercontent.com/20023170uhi/20023170_DataAnalytics/main/lnrdata.csv', index_col=0)
df2 = pd.read_csv('https://raw.githubusercontent.com/20023170uhi/20023170_DataAnalytics/main/lnrdata2.csv', index_col=0)

dnndf1 = pd.read_csv('https://raw.githubusercontent.com/20023170uhi/20023170_DataAnalytics/main/dnndata1.csv', index_col=0)
dnndf2 = pd.read_csv('https://raw.githubusercontent.com/20023170uhi/20023170_DataAnalytics/main/dnndata2.csv', index_col=0)


TensorFlow 1.x selected.
1.15.2


In [ ]:
print(df1[:6])
print(df2[:6])

   weekday_sun  season  temp  dewp  NUM_COLLISIONS
1            4       1  38.0  25.6        0.000000
2            5       1  27.5  12.1        0.278873
3            6       1  21.8   7.8        0.473239
4            7       1  32.2  21.1        0.349296
5            2       1  37.3  24.5        0.022535
6            1       1  35.7  31.3        0.033803
   weekday_sun  season  temp  dewp  NUM_COLLISIONS
1            4       1  38.0  25.6        0.000000
2            5       1  27.5  12.1        0.274238
3            6       1  21.8   7.8        0.465374
4            7       1  32.2  21.1        0.343490
5            2       1  37.3  24.5        0.022161
6            1       1  35.7  31.3        0.033241


In [ ]:
print(dnndf1[:6])
print(dnndf2[:6])

   Apr  Aug  Dec  Feb  Jan  Jul  ...  year  temp  wdsp  sndp  fog  NUM_COLLISIONS
1    0    0    0    0    1    0  ...  2013  38.0  15.0   0.0    0        0.000000
2    0    0    0    0    1    0  ...  2013  27.5  12.4   0.0    0        0.278873
3    0    0    0    0    1    0  ...  2013  21.8   9.8   0.0    0        0.473239
4    0    0    0    0    1    0  ...  2013  32.2  11.3   0.0    0        0.349296
5    0    0    0    0    1    0  ...  2013  37.3  13.2   0.0    0        0.022535
6    0    0    0    0    1    0  ...  2013  35.7   5.9   0.0    0        0.033803

[6 rows x 33 columns]
   Apr  Aug  Dec  Feb  Jan  Jul  ...  year  temp  wdsp  sndp  fog  NUM_COLLISIONS
1    0    0    0    0    1    0  ...  2013  38.0  15.0   0.0    0        0.000000
2    0    0    0    0    1    0  ...  2013  27.5  12.4   0.0    0        0.274238
3    0    0    0    0    1    0  ...  2013  21.8   9.8   0.0    0        0.465374
4    0    0    0    0    1    0  ...  2013  32.2  11.3   0.0    0        0.

## LNR Data 1

### LNR all variables

LinearRegression has RMSE of 0.17614062325282587
Just using average = 0.506931287930208 has RMSE of 0.21959495431642645  


In [ ]:
shuffle = df1.iloc[np.random.permutation(len(df1))]
predictors = shuffle.iloc[:,0:4]
print(predictors[:6])

      weekday_sun  season  temp  dewp
1537            5       2  53.9  49.6
223             7       4  67.4  59.7
481             5       2  51.2  44.3
1209            6       4  59.9  46.7
996             4       3  46.7  36.2
77              6       2  38.3  29.9


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
1537,5,2,53.9,49.6,0.527273
223,7,4,67.4,59.7,0.836620
481,5,2,51.2,44.3,0.713115
1209,6,4,59.9,46.7,0.680739
996,4,3,46.7,36.2,0.568238


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

1537    0.527273
223     0.836620
481     0.713115
1209    0.680739
996     0.568238
77      0.357746
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 4
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb55d17828>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running l

### LNR Weekday  

LinearRegression has RMSE of 0.193136152893102
Just using average = 0.5039372956128163 has RMSE of 0.22209884639004893.  


In [ ]:
shuffle = df1.iloc[np.random.permutation(len(df1))]
predictors = shuffle.iloc[:,0]
print(predictors[:6])

1055    6
814     2
620     6
740     2
1106    7
1440    6
Name: weekday_sun, dtype: int64


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
1055,6,2,46.5,41.8,0.310174
814,2,2,43.7,27.5,0.471464
620,6,3,59.9,56.3,0.494536
740,2,1,27.4,16.0,0.116625
1106,7,1,40.2,23.9,0.126649


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

1055    0.310174
814     0.471464
620     0.494536
740     0.116625
1106    0.126649
1440    0.179221
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 1
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model_w', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_w', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb5b8b4780>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_w', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running

### LNR Season  

LinearRegression has RMSE of 0.21857136416783593
Just using average = 0.5089524497729809 has RMSE of 0.22073434760287095

In [ ]:
shuffle = df1.iloc[np.random.permutation(len(df1))]
predictors = shuffle.iloc[:,1]
print(predictors[:6])

139     2
318     3
1123    2
1457    1
997     3
1655    3
Name: season, dtype: int64


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
139,6,2,61.5,57.6,0.997183
318,5,3,38.0,21.5,0.636620
1123,4,2,48.3,45.7,0.474934
1457,3,1,39.1,25.5,0.264935
997,5,3,53.8,46.9,0.712159


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

139     0.997183
318     0.636620
1123    0.474934
1457    0.264935
997     0.712159
1655    0.748052
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 1
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model_s', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_s', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb5559eb70>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_s', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running

### LNR Temperature  

LinearRegression has RMSE of 0.21548913456651844
Just using average = 0.503537721138607 has RMSE of 0.21813183732023203  



In [ ]:
shuffle = df1.iloc[np.random.permutation(len(df1))]
predictors = shuffle.iloc[:,2]
print(predictors[:6])

1622    70.5
402     41.4
1537    53.9
439     37.1
162     61.3
79      38.5
Name: temp, dtype: float64


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
1622,4,4,70.5,69.8,0.638961
402,2,1,41.4,36.5,0.573770
1537,5,2,53.9,49.6,0.527273
439,4,2,37.1,29.8,0.469945
162,1,4,61.3,57.0,0.270423


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

1622    0.638961
402     0.573770
1537    0.527273
439     0.469945
162     0.270423
79      0.301408
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 1
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model_t', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_t', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb5571fa90>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_t', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running

### LNR Dew Point  

LinearRegression has RMSE of 0.2026404257267636
Just using average = 0.5073890080818348 has RMSE of 0.20708710813799586  


In [ ]:
shuffle = df1.iloc[np.random.permutation(len(df1))]
predictors = shuffle.iloc[:,3]
print(predictors[:6])

1279    62.1
1292    63.2
708      8.3
1508    44.5
1699    39.9
1690    44.2
Name: dewp, dtype: float64


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
1279,2,4,69.1,62.1,0.398417
1292,7,3,69.2,63.2,0.709763
708,5,2,26.9,8.3,0.229508
1508,2,2,46.7,44.5,0.293506
1699,6,3,48.3,39.9,0.685714


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

1279    0.398417
1292    0.709763
708     0.229508
1508    0.293506
1699    0.685714
1690    0.670130
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 1
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model_d', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_d', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb4d84def0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_d', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running

### LNR Weekday/Season/Temperature  

LinearRegression has RMSE of 0.19166140928282666
Just using average = 0.5087825185751887 has RMSE of 0.2152122587226192

In [ ]:
shuffle = df1.iloc[np.random.permutation(len(df1))]
predictors = shuffle.iloc[:,0:3]
print(predictors[:6])

      weekday_sun  season  temp
1499            6       2  49.8
1293            2       3  63.2
14              4       1  42.3
1420            4       1  40.0
588             1       4  69.1
1249            6       4  70.4


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
1499,6,2,49.8,38.5,0.394805
1293,2,3,63.2,57.5,0.184697
14,4,1,42.3,38.3,0.380282
1420,4,1,40.0,30.9,0.568831
588,1,4,69.1,65.6,0.322404


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

1499    0.394805
1293    0.184697
14      0.380282
1420    0.568831
588     0.322404
1249    0.799472
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 3
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model_wst', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_wst', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb5b7270b8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_wst', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runni

### LNR Weekday/Temperature/Dew Point  

LinearRegression has RMSE of 0.1725977316174406
Just using average = 0.510238184419735 has RMSE of 0.2110724583122209

In [ ]:
shuffle = df1.iloc[np.random.permutation(len(df1))]
predictors = shuffle.iloc[:, [0, 2, 3]]
print(predictors[:6])

      weekday_sun  temp  dewp
222             6  67.2  52.7
296             3  50.7  33.0
1471            3  21.9   6.4
501             4  58.2  47.9
1301            3  65.1  52.1
140             7  61.0  58.7


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
222,6,4,67.2,52.7,0.614085
296,3,3,50.7,33.0,0.673239
1471,3,2,21.9,6.4,0.919481
501,4,4,58.2,47.9,0.781421
1301,3,3,65.1,52.1,0.580475


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

222     0.614085
296     0.673239
1471    0.919481
501     0.781421
1301    0.580475
140     0.808451
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 3
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model_wtd', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_wtd', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb557de4e0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_wtd', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runni

### LNR Weekday/Season/Dew Point  

LinearRegression has RMSE of 0.19356291642322979
Just using average = 0.5080321250164174 has RMSE of 0.21319484832817295

In [ ]:
shuffle = df1.iloc[np.random.permutation(len(df1))]
predictors = shuffle.iloc[:,[0, 1, 3]]
print(predictors[:6])

      weekday_sun  season  dewp
606             6       3  47.4
1744            5       2   5.0
410             3       2  20.3
1298            7       3  68.3
1080            6       1  24.4
1222            6       4  59.4


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
606,6,3,61.1,47.4,0.745902
1744,5,2,21.9,5.0,0.361039
410,3,2,27.3,20.3,0.172131
1298,7,3,71.4,68.3,0.757256
1080,6,1,35.0,24.4,0.667546


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

606     0.745902
1744    0.361039
410     0.172131
1298    0.757256
1080    0.667546
1222    0.567282
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 3
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model_wsd', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_wsd', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb5dcdc710>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_wsd', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runni

### LNR Weekday/Season  

LinearRegression has RMSE of 0.1725977316174406
Just using average = 0.510238184419735 has RMSE of 0.2110724583122209

In [ ]:
shuffle = df1.iloc[np.random.permutation(len(df1))]
predictors = shuffle.iloc[:,0:2]
print(predictors[:6])

      weekday_sun  season
680             3       2
482             6       2
1417            2       1
1066            5       1
176             1       4
1566            1       4


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
680,3,2,53.2,49.0,0.729508
482,6,2,57.8,51.8,0.770492
1417,2,1,29.8,13.4,0.148052
1066,5,1,32.9,18.2,0.467018
176,1,4,65.4,64.2,0.118310


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

680     0.729508
482     0.770492
1417    0.148052
1066    0.467018
176     0.118310
1566    0.394805
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 2
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model_ws', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_ws', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb558e5e48>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_ws', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runnin

### LNR Weekday/Temperature  

LinearRegression has RMSE of 0.18247544862031528
Just using average = 0.5033623596718003 has RMSE of 0.22395551470404063

In [ ]:
shuffle = df1.iloc[np.random.permutation(len(df1))]
predictors = shuffle.iloc[:,[0, 2]]
print(predictors[:6])

      weekday_sun  temp
1565            2  67.2
189             2  67.9
973             1  56.2
841             7  53.8
231             2  66.2
560             1  66.4


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
1565,2,4,67.2,65.1,0.818182
189,2,4,67.9,66.3,0.546479
973,1,3,56.2,46.7,0.332506
841,7,2,53.8,48.0,0.806452
231,2,4,66.2,55.2,0.411268


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

1565    0.818182
189     0.546479
973     0.332506
841     0.806452
231     0.411268
560     0.270492
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 2
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model_wt', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_wt', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb4d7aec50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_wt', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runnin

### LNR Weekday/Dew Point  

LinearRegression has RMSE of 0.1822486840081702
Just using average = 0.5117637095613321 has RMSE of 0.22387599018335627

In [ ]:
shuffle = df1.iloc[np.random.permutation(len(df1))]
predictors = shuffle.iloc[:,[0, 3]]
print(predictors[:6])

      weekday_sun  dewp
1054            5  40.6
790             5  24.8
426             5  23.6
352             2  29.8
1534            1  37.3
912             4  71.7


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
1054,5,2,44.8,40.6,0.245658
790,5,2,37.9,24.8,0.610422
426,5,2,35.0,23.6,0.404372
352,2,2,39.3,29.8,0.264789
1534,1,2,52.7,37.3,0.467532


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

1054    0.245658
790     0.610422
426     0.404372
352     0.264789
1534    0.467532
912     0.655087
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 2
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model_wd', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_wd', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb5b8b4e10>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_wd', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runnin

## LNR Data 2

### LNR2 all variables  

LinearRegression has RMSE of 0.18264648695795516
Just using average = 0.51860027961461 has RMSE of 0.21034197462434914

In [ ]:
shuffle = df2.iloc[np.random.permutation(len(df2))]
predictors = shuffle.iloc[:,0:4]
print(predictors[:6])

      weekday_sun  season  temp  dewp
1274            7       4  74.3  72.2
299             5       3  46.8  38.6
619             2       3  63.5  55.3
663             3       3  48.2  42.2
933             4       4  72.8  68.7
515             6       4  61.0  53.0


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
1274,7,4,74.3,72.2,0.769752
299,5,3,46.8,38.6,0.490305
619,2,3,63.5,55.3,0.609290
663,3,3,48.2,42.2,0.500000
933,4,4,72.8,68.7,0.518817


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

1274    0.769752
299     0.490305
619     0.609290
663     0.500000
933     0.518817
515     0.508197
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 4
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model2', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model2', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb559640b8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model2', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running 

### LNR2 Weekday  

LinearRegression has RMSE of 0.18700561386254116
Just using average = 0.5211066931727971 has RMSE of 0.21307049870186248 


In [ ]:
shuffle = df2.iloc[np.random.permutation(len(df2))]
predictors = shuffle.iloc[:,0]
print(predictors[:6])

258     6
1588    1
542     7
276     3
1678    2
250     5
Name: weekday_sun, dtype: int64


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
258,6,3,62.4,54.6,0.429363
1588,1,4,68.7,67.4,0.298405
542,7,4,69.1,62.4,0.724044
276,3,3,65.7,62.3,0.656510
1678,2,3,56.3,50.9,0.328018


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

258     0.429363
1588    0.298405
542     0.724044
276     0.656510
1678    0.328018
250     0.540166
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 1
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model2_w', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model2_w', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb55658d30>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model2_w', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runnin

### LNR2 Season  

LinearRegression has RMSE of 0.21969390879839978
Just using average = 0.5200612879303381 has RMSE of 0.2308899460317336

In [ ]:
shuffle = df2.iloc[np.random.permutation(len(df2))]
predictors = shuffle.iloc[:,1]
print(predictors[:6])

1108    1
1193    2
410     1
1541    2
291     3
357     2
Name: season, dtype: int64


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
1108,1,1,42.8,31.7,0.273138
1193,3,2,59.4,50.3,0.595937
410,5,1,25.8,12.6,0.366120
1541,2,2,47.9,41.7,0.630979
291,4,3,59.1,52.4,0.440443


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

1108    0.273138
1193    0.595937
410     0.366120
1541    0.630979
291     0.440443
357     0.263158
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 1
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model2_s', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model2_s', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb558bf128>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model2_s', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runnin

### LNR2 Temperature  

LinearRegression has RMSE of 0.247043336982424
Just using average = 0.5167012118479363 has RMSE of 0.22569156382665287

In [ ]:
shuffle = df2.iloc[np.random.permutation(len(df2))]
predictors = shuffle.iloc[:,2]
print(predictors[:6])

1109    47.8
1634    68.0
796     41.6
309     41.7
1686    65.8
70      37.0
Name: temp, dtype: float64


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
1109,3,1,47.8,40.9,0.514673
1634,6,4,68.0,60.8,0.530752
796,1,2,41.6,27.3,0.053763
309,2,3,41.7,22.4,0.526316
1686,1,3,65.8,65.6,0.261959


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

1109    0.514673
1634    0.530752
796     0.053763
309     0.526316
1686    0.261959
70      0.487535
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 1
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model2_t', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model2_t', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb55827668>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model2_t', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runnin

### LNR2 Dew Point  

LinearRegression has RMSE of 0.21996257870330269
Just using average = 0.5150669142147675 has RMSE of 0.2224265992045367

In [ ]:
shuffle = df2.iloc[np.random.permutation(len(df2))]
predictors = shuffle.iloc[:,3]
print(predictors[:6])

173     64.6
629     57.0
127     51.0
544     63.6
1301    66.8
322     41.9
Name: dewp, dtype: float64


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
173,5,4,69.5,64.6,0.664820
629,4,3,63.0,57.0,0.663934
127,7,2,56.1,51.0,0.878116
544,1,4,68.7,63.6,0.169399
1301,6,3,68.5,66.8,0.742664


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

173     0.664820
629     0.663934
127     0.878116
544     0.169399
1301    0.742664
322     0.889197
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 1
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model2_d', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model2_d', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb5d37b278>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model2_d', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runnin

### LNR2 Weekday/Season/Temperature  

LinearRegression has RMSE of 0.18370707716336826
Just using average = 0.5191582984332717 has RMSE of 0.21631631983459546

In [ ]:
shuffle = df2.iloc[np.random.permutation(len(df2))]
predictors = shuffle.iloc[:,0:3]
print(predictors[:6])

      weekday_sun  season  temp
1720            1       3  35.8
526             4       4  60.0
1023            3       3  40.5
1739            7       2  48.2
846             3       2  58.9
28              5       1  45.1


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
1720,1,3,35.8,29.8,0.302961
526,4,4,60.0,51.9,0.696721
1023,3,3,40.5,26.7,0.561828
1739,7,2,48.2,44.2,0.833713
846,3,2,58.9,52.6,0.048387


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

1720    0.302961
526     0.696721
1023    0.561828
1739    0.833713
846     0.048387
28      0.321330
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 3
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model2_wst', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model2_wst', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb558bc2b0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model2_wst', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runn

### LNR2 Weekday/Temperature/Dew Point  

LinearRegression has RMSE of 0.1674970718624497
Just using average = 0.5134672489489462 has RMSE of 0.2080217598733402

In [ ]:
shuffle = df2.iloc[np.random.permutation(len(df2))]
predictors = shuffle.iloc[:, [0, 2, 3]]
print(predictors[:6])

      weekday_sun  temp  dewp
1029            1  45.4  40.9
291             4  59.1  52.4
801             7  43.9  41.7
894             7  64.9  57.6
1619            5  68.6  66.6
1733            2  50.1  47.2


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
1029,1,2,45.4,40.9,0.223118
291,4,3,59.1,52.4,0.440443
801,7,2,43.9,41.7,0.225806
894,7,4,64.9,57.6,0.690860
1619,5,4,68.6,66.6,0.482916


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

1029    0.223118
291     0.440443
801     0.225806
894     0.690860
1619    0.482916
1733    0.113895
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 3
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model2_wtd', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model2_wtd', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb55652fd0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model2_wtd', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runn

### LNR2 Weekday/Season/Dew Point  

LinearRegression has RMSE of 0.17953476541335797
Just using average = 0.5193390691199882 has RMSE of 0.22505355155874024  

In [ ]:
shuffle = df2.iloc[np.random.permutation(len(df2))]
predictors = shuffle.iloc[:,[0, 1, 3]]
print(predictors[:6])

      weekday_sun  season  dewp
1224            6       4  59.4
1138            4       2  34.9
639             7       3  59.7
1516            3       2  48.3
268             1       3  50.1
643             4       3  52.3


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
1224,6,4,63.0,59.4,0.575621
1138,4,2,48.6,34.9,0.665914
639,7,3,63.2,59.7,0.740437
1516,3,2,56.4,48.3,0.186788
268,1,3,54.9,50.1,0.346260


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

1224    0.575621
1138    0.665914
639     0.740437
1516    0.186788
268     0.346260
643     0.628415
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 3
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model2_wsd', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model2_wsd', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb558ad320>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model2_wsd', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runn

### LNR2 Weekday/Season  

LinearRegression has RMSE of 0.18692129118233317
Just using average = 0.5063489271925886 has RMSE of 0.2249993352316868  



In [ ]:
shuffle = df1.iloc[np.random.permutation(len(df1))]
predictors = shuffle.iloc[:,0:2]
print(predictors[:6])

      weekday_sun  season
1543            5       2
341             3       2
747             1       1
1738            5       2
1068            7       1
551             7       4


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
1543,5,2,55.4,53.1,0.600000
341,3,2,32.0,15.5,0.729577
747,1,1,25.7,21.0,0.129032
1738,5,2,44.3,31.7,0.680519
1068,7,1,36.5,25.6,0.775726


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

1543    0.600000
341     0.729577
747     0.129032
1738    0.680519
1068    0.775726
551     0.754098
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 2
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model2_ws', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model2_ws', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb55b35ac8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model2_ws', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runni

### LNR2 Weekday/Temperature  

LinearRegression has RMSE of 0.19042619876016584
Just using average = 0.5120928705334473 has RMSE of 0.22171086182018704  

In [ ]:
shuffle = df2.iloc[np.random.permutation(len(df2))]
predictors = shuffle.iloc[:,[0, 2]]
print(predictors[:6])

      weekday_sun  temp
715             3  44.0
658             5  57.7
552             3  65.9
240             1  70.7
281             2  58.6
1122            1  50.1


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
715,3,1,44.0,32.1,0.258065
658,5,3,57.7,52.4,0.483607
552,3,4,65.9,63.3,0.491803
240,1,3,70.7,68.3,0.274238
281,2,3,58.6,53.4,0.484765


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

715     0.258065
658     0.483607
552     0.491803
240     0.274238
281     0.484765
1122    0.106095
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 2
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model2_wt', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model2_wt', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb4d87df60>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model2_wt', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runni

### LNR2 Weekday/Dew Point  

LinearRegression has RMSE of 0.17490662132852705
Just using average = 0.5205753974336695 has RMSE of 0.2169003001547654

In [ ]:
shuffle = df2.iloc[np.random.permutation(len(df2))]
predictors = shuffle.iloc[:,[0, 3]]
print(predictors[:6])

      weekday_sun  dewp
288             2  45.0
842             6  41.1
902             2  60.2
368             2  43.0
1415            4  40.7
623             5  60.6


In [ ]:
shuffle[:5]

,weekday_sun,season,temp,dewp,NUM_COLLISIONS
288,2,3,57.3,45.0,0.249307
842,6,2,54.8,41.1,0.572581
902,2,4,68.5,60.2,0.131720
368,2,1,44.8,43.0,0.240437
1415,4,1,44.3,40.7,0.576310


In [ ]:
targets = shuffle.iloc[:,4]
print(targets[:6])

288     0.249307
842     0.572581
902     0.131720
368     0.240437
1415    0.576310
623     0.497268
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 2
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/linear_regression_trained_model2_wd', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model2_wd', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb4d88e9b0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model2_wd', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runni

## DNN Data 1

###  DNN1 all variables  

DNNRegression has RMSE of 0.1495475916918793
Just using average = 0.5057197010182095 has RMSE of 0.21762087224134297

In [ ]:
shuffle = dnndf1.iloc[np.random.permutation(len(dnndf1))]
predictors = shuffle.iloc[:,0:32]
print(predictors[:6])

      Apr  Aug  Dec  Feb  Jan  Jul  ...  gust  year  temp  wdsp  sndp  fog
455     1    0    0    0    0    0  ...  28.9  2014  37.5  15.2   0.0    0
541     0    0    0    0    0    1  ...  22.0  2014  71.4  13.1   0.0    1
1557    0    0    0    0    0    0  ...   0.0  2017  57.2   7.5   0.0    0
1366    0    0    0    0    0    0  ...  36.9  2016  47.0  14.4   0.0    1
393     0    0    0    1    0    0  ...  18.1  2014  24.0   9.2   5.1    0
1403    0    0    1    0    0    0  ...  33.0  2016  37.3  10.3   0.0    0

[6 rows x 32 columns]


In [ ]:
shuffle[:5]

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,Fri,Mon,Sat,Sun,Thu,Tue,Wed,Autumn,Spring,Summer,Winter,mxpsd,dewp,prcp,gust,year,temp,wdsp,sndp,fog,NUM_COLLISIONS
455,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,21.0,23.8,0.67,28.9,2014,37.5,15.2,0.0,0,0.357923
541,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,15.9,69.1,0.00,22.0,2014,71.4,13.1,0.0,1,0.573770
1557,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,14.0,45.8,0.00,0.0,2017,57.2,7.5,0.0,0,0.864935
1366,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,27.0,42.3,0.13,36.9,2016,47.0,14.4,0.0,1,0.266491
393,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,15.0,6.4,0.00,18.1,2014,24.0,9.2,5.1,0,0.707650


In [ ]:
targets = shuffle.iloc[:, 32]
print(targets[:6])

455     0.357923
541     0.573770
1557    0.864935
1366    0.266491
393     0.707650
1403    0.300792
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 32
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/DNN_house_regression_trained_model', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_house_regression_trained_model', hidden_units=[20,18,14], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb55b0cc18>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_house_regression_trained_model', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runnin

### DNN1 4 variables  

DNNRegression has RMSE of 0.16791662773573895
Just using average = 0.5047165186097293 has RMSE of 0.22351629988633476



In [ ]:
shuffle = dnndf1.iloc[np.random.permutation(len(dnndf1))]
predictors = shuffle.iloc[:,[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 28]]
print(predictors[:6])

      Fri  Mon  Sat  Sun  Thu  Tue  ...  Autumn  Spring  Summer  Winter  dewp  temp
546     0    0    0    1    0    0  ...       0       0       1       0  57.9  65.2
240     0    1    0    0    0    0  ...       1       0       0       0  68.7  70.8
1347    0    0    0    0    0    1  ...       1       0       0       0  37.1  46.2
1589    0    0    0    0    1    0  ...       0       0       1       0  70.6  74.7
371     1    0    0    0    0    0  ...       0       0       0       1  35.8  38.6
463     1    0    0    0    0    0  ...       0       1       0       0  32.3  49.0

[6 rows x 13 columns]


In [ ]:
shuffle[:5]

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,Fri,Mon,Sat,Sun,Thu,Tue,Wed,Autumn,Spring,Summer,Winter,mxpsd,dewp,prcp,gust,year,temp,wdsp,sndp,fog,NUM_COLLISIONS
546,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,13.0,57.9,0.00,18.1,2014,65.2,10.0,0.0,0,0.202186
240,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,11.1,68.7,0.00,0.0,2013,70.8,7.7,0.0,1,0.140845
1347,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,8.0,37.1,0.01,0.0,2016,46.2,4.4,0.0,0,0.802111
1589,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,12.0,70.6,0.00,0.0,2017,74.7,7.7,0.0,1,0.872727
371,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,8.9,35.8,0.00,0.0,2014,38.6,3.7,0.0,1,0.579235


In [ ]:
targets = shuffle.iloc[:, 32]
print(targets[:6])

546     0.202186
240     0.140845
1347    0.802111
1589    0.872727
371     0.579235
463     0.655738
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 13
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/DNN_house_regression_trained_model_wstd', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_house_regression_trained_model_wstd', hidden_units=[20,18,14], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb5d37b6d8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_house_regression_trained_model_wstd', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:R

## DNN Data 2

###  DNN2 all variables  

DNNRegression has RMSE of 0.1723163919584417
Just using average = 0.5168076771448048 has RMSE of 0.21937121029012974


In [ ]:
shuffle = dnndf2.iloc[np.random.permutation(len(dnndf2))]
predictors = shuffle.iloc[:,0:32]
print(predictors[:6])

      Apr  Aug  Dec  Feb  Jan  Jul  ...  gust  year  temp  wdsp  sndp  fog
1197    0    0    0    0    0    0  ...   0.0  2016  60.0   8.2   0.0    1
858     0    0    0    0    0    0  ...   0.0  2015  59.5   7.7   0.0    0
1628    0    1    0    0    0    0  ...   0.0  2017  67.6   5.8   0.0    1
1665    0    0    0    0    0    0  ...   0.0  2017  64.7   3.4   0.0    1
1727    0    0    0    0    0    0  ...  41.0  2017  54.8  16.8   0.0    0
1528    1    0    0    0    0    0  ...  27.0  2017  53.4  12.6   0.0    0

[6 rows x 32 columns]


In [ ]:
shuffle[:5]

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,Fri,Mon,Sat,Sun,Thu,Tue,Wed,Autumn,Spring,Summer,Winter,mxpsd,dewp,prcp,gust,year,temp,wdsp,sndp,fog,NUM_COLLISIONS
1197,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,12.0,55.8,0.00,0.0,2016,60.0,8.2,0.0,1,0.855530
858,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,11.1,45.8,0.14,0.0,2015,59.5,7.7,0.0,0,0.422043
1628,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,8.0,65.8,0.00,0.0,2017,67.6,5.8,0.0,1,0.567198
1665,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,8.9,64.2,0.00,0.0,2017,64.7,3.4,0.0,1,0.410023
1727,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,31.1,50.4,0.00,41.0,2017,54.8,16.8,0.0,0,0.380410


In [ ]:
targets = shuffle.iloc[:, 32]
print(targets[:6])

1197    0.855530
858     0.422043
1628    0.567198
1665    0.410023
1727    0.380410
1528    0.145786
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 32
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/DNN_house_regression_trained_model2', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_house_regression_trained_model2', hidden_units=[20,18,14], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb4d843e10>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_house_regression_trained_model2', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runni

### DNN2 4 variables  

DNNRegression has RMSE of 0.16338602227240437
Just using average = 0.5197966183645708 has RMSE of 0.2115608324356858  




In [ ]:
shuffle = dnndf2.iloc[np.random.permutation(len(dnndf2))]
predictors = shuffle.iloc[:,[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 28]]
print(predictors[:6])

      Fri  Mon  Sat  Sun  Thu  Tue  ...  Autumn  Spring  Summer  Winter  dewp  temp
585     0    0    1    0    0    0  ...       0       0       1       0  60.1  65.6
912     0    0    0    0    0    1  ...       0       0       1       0  71.7  75.0
781     0    0    1    0    0    0  ...       0       1       0       0  31.0  34.2
1700    0    1    0    0    0    0  ...       1       0       0       0  58.7  59.5
1725    1    0    0    0    0    0  ...       1       0       0       0  33.2  43.8
1098    0    0    0    0    1    0  ...       0       0       0       1  23.1  36.0

[6 rows x 13 columns]


In [ ]:
shuffle[:5]

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,Fri,Mon,Sat,Sun,Thu,Tue,Wed,Autumn,Spring,Summer,Winter,mxpsd,dewp,prcp,gust,year,temp,wdsp,sndp,fog,NUM_COLLISIONS
585,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,18.1,60.1,0.02,22.9,2014,65.6,10.8,0.0,0,0.210383
912,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,15.0,71.7,0.00,22.0,2015,75.0,10.4,0.0,0,0.709677
781,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,13.0,31.0,0.02,0.0,2015,34.2,8.5,1.2,0,0.282258
1700,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,12.0,58.7,0.00,0.0,2017,59.5,7.2,0.0,1,0.599089
1725,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,22.9,33.2,0.55,35.9,2017,43.8,15.6,0.0,0,0.929385


In [ ]:
targets = shuffle.iloc[:, 32]
print(targets[:6])

585     0.210383
912     0.709677
781     0.282258
1700    0.599089
1725    0.929385
1098    0.460497
Name: NUM_COLLISIONS, dtype: float64


In [ ]:
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
trainsize = int(len(shuffle['NUM_COLLISIONS'])*0.8)
testsize = len(shuffle['NUM_COLLISIONS']) - trainsize

nppredictors = 13
noutputs = 1

In [ ]:
shutil.rmtree('/tmp/DNN_house_regression_trained_model2_wstd', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_house_regression_trained_model2_wstd', hidden_units=[20,18,14], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

print("starting to train");

estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_COLLISIONS, steps=10000)

preds = estimator.predict(x=predictors[trainsize:].values)

predslistscale = preds['scores']*SCALE_NUM_COLLISIONS

rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));

avg = np.mean(shuffle['NUM_COLLISIONS'][:trainsize])

rmse = np.sqrt(np.mean((shuffle['NUM_COLLISIONS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb55e38390>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_house_regression_trained_model2_wstd', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:

# Results

In [ ]:
tdf1 = pd.read_csv('https://raw.githubusercontent.com/20023170uhi/20023170_DataAnalytics/main/tsample1.csv', index_col=0, )
tdf2 = pd.read_csv('https://raw.githubusercontent.com/20023170uhi/20023170_DataAnalytics/main/tsample2.csv', index_col=0, )

In [ ]:
tdf1

,weekday_sat,weekday_sun,dayindex,dayofyear,day,year,mo,da,collision_date,temp,dewp,slp,visib,wdsp,mxpsd,gust,max,min,prcp,fog,NUM_COLLISIONS,season,sndp
1,5,5,303,303,3,2013,10,30,2013-10-30,46.8,38.6,1025.8,10.0,5.2,11.1,0.0,57.9,34.0,0.00,0,0.498592,3,0.0
2,3,3,497,132,1,2014,5,12,2014-05-12,57.8,46.2,1018.5,9.8,9.1,13.0,17.1,70.0,48.9,0.23,0,0.557377,2,0.0
3,4,4,1541,80,2,2017,3,21,2017-03-21,39.9,26.4,1011.2,10.0,7.9,11.1,0.0,50.0,34.0,0.00,0,0.514286,2,0.0
4,3,3,413,48,1,2014,2,17,2014-02-17,24.3,8.9,1019.8,9.3,13.1,19.0,28.0,32.0,19.0,0.00,0,0.311475,1,9.1
5,7,7,1656,195,5,2017,7,14,2017-07-14,63.7,61.5,1018.4,9.0,8.9,13.0,19.0,80.1,62.1,0.00,0,0.698701,4,0.0


In [ ]:
tdf2

,weekday_sat,weekday_sun,dayindex,dayofyear,day,year,mo,da,collision_date,temp,dewp,slp,visib,wdsp,mxpsd,gust,max,min,prcp,fog,NUM_COLLISIONS,season,sndp
1,1,2,1755,294,6,2017,10,21,2017-10-21,58.8,50.3,1024.1,10.0,7.0,12.0,0,68.0,46.9,0.00,0,0.403189,3,0.0
2,5,5,1472,11,3,2017,1,11,2017-01-11,45.0,40.9,1027.3,6.5,15.1,27.0,34,50.0,28.9,0.33,0,0.507973,1,1.2
3,5,5,786,56,3,2015,2,25,2015-02-25,29.9,21.8,1009.4,7.9,12.4,20.0,27,33.8,23.0,0.02,0,0.607527,1,2.0
4,4,4,603,238,2,2014,8,26,2014-08-26,63.0,56.0,1021.1,10.0,3.5,8.9,0,72.0,53.1,0.00,0,0.631148,4,0.0
5,6,6,927,197,4,2015,7,16,2015-07-16,67.3,57.3,1013.9,9.2,16.6,22.0,28,82.0,60.1,0.00,0,0.763441,4,0.0


## Validating LNR

In [ ]:
input = tdf1[["weekday_sun", "season", "temp", "dewp"]]

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)
predslistnorm = preds['scores']
predslistscale = preds['scores']*825
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb5d33f198>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model/model.ck

In [ ]:
input = tdf1[["weekday_sun", "temp", "dewp"]]

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_wtd', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)
predslistnorm = preds['scores']
predslistscale = preds['scores']*825
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb55cf70b8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_wtd', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_wtd/

In [ ]:
input = tdf1[["weekday_sun", "season"]]

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model_ws', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)
predslistnorm = preds['scores']
predslistscale = preds['scores']*825
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb55cad5f8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model_ws', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model_ws/mo

In [ ]:
input = tdf2[["weekday_sun", "season", "temp", "dewp"]]

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model2', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)
predslistnorm = preds['scores']
predslistscale = preds['scores']*850
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb554f02b0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model2', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model2/model.

In [ ]:
input = tdf2[["weekday_sun", "temp", "dewp"]]

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model2_wtd', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)
predslistnorm = preds['scores']
predslistscale = preds['scores']*850
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb5ba0d6d8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model2_wtd', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model2_wt

In [ ]:
input = tdf2[["weekday_sun", "dewp"]]

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model2_wd', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)
predslistnorm = preds['scores']
predslistscale = preds['scores']*850
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb55a037b8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model2_wd', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model2_wd/

## Validating DNN

In [ ]:
for col in dnndf1.columns:
  print(col)

Apr
Aug
Dec
Feb
Jan
Jul
Jun
Mar
May
Nov
Oct
Sep
Fri
Mon
Sat
Sun
Thu
Tue
Wed
Autumn
Spring
Summer
Winter
mxpsd
dewp
prcp
gust
year
temp
wdsp
sndp
fog
NUM_COLLISIONS


In [ ]:
dnntdf1 = pd.DataFrame.from_dict(data = 
				{
         'Apr' : [0,0,0,0,0],
         'Aug' : [0,0,0,0,0],
         'Dec' : [0,0,0,0,0],
         'Feb' : [0,0,0,1,0],
         'Jan' : [0,0,0,0,0],
         'Jul' : [0,0,0,0,1],
         'Jun' : [0,0,0,0,0],
         'Mar' : [0,0,1,0,0],
         'May' : [0,1,0,0,0],
         'Nov' : [0,0,0,0,0],
         'Oct' : [1,0,0,0,0],
         'Sep' : [0,0,0,0,0],
         'Fri' : [0,0,0,0,1],
         'Mon' : [0,1,0,1,0],
         'Sat' : [0,0,0,0,0],
         'Sun' : [0,0,0,0,0],
         'Thu' : [0,0,0,0,0],
         'Tue' : [0,0,1,0,0],
         'Wed' : [1,0,0,0,0],
         'Autumn' : [1,0,0,0,0],
         'Spring' : [0,1,1,0,0],
         'Summer' : [0,0,0,0,1],
         'Winter' : [0,0,0,1,0],
         'mxpsd' : [11.1,13.0,11.1,19.0,13.0], 
         'dewp' : [38.6,46.2,26.4,8.9,61.5], 
         'prcp' : [0,0.23,0,0,0],
         'gust' : [0,17.1,0,28,19],
         'year' : [2013,2014,2017,2014,2017],
         'temp' : [46.8,57.8,39.9,24.3,63.7],
         'wdsp' : [5.2,9.1,7.9,13.1,8.9],
         'sndp' : [0,0,0,9.1,0],
         'fog' : [0,0,0,0,0]
        })

In [ ]:
dnntdf1

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,Fri,Mon,Sat,Sun,Thu,Tue,Wed,Autumn,Spring,Summer,Winter,mxpsd,dewp,prcp,gust,year,temp,wdsp,sndp,fog
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,11.1,38.6,0.00,0.0,2013,46.8,5.2,0.0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,13.0,46.2,0.23,17.1,2014,57.8,9.1,0.0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,11.1,26.4,0.00,0.0,2017,39.9,7.9,0.0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,19.0,8.9,0.00,28.0,2014,24.3,13.1,9.1,0
4,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,13.0,61.5,0.00,19.0,2017,63.7,8.9,0.0,0


In [ ]:
dnntdf2 = pd.DataFrame.from_dict(data = 
				{
         'Apr' : [0,0,0,0,0],
         'Aug' : [0,0,0,1,0],
         'Dec' : [0,0,0,0,0],
         'Feb' : [0,0,1,0,0],
         'Jan' : [0,1,0,0,0],
         'Jul' : [0,0,0,0,1],
         'Jun' : [0,0,0,0,0],
         'Mar' : [0,0,0,0,0],
         'May' : [0,0,0,0,0],
         'Nov' : [0,0,0,0,0],
         'Oct' : [1,0,0,0,0],
         'Sep' : [0,0,0,0,0],
         'Fri' : [0,0,0,0,0],
         'Mon' : [0,0,0,0,0],
         'Sat' : [1,0,0,0,0],
         'Sun' : [0,0,0,0,0],
         'Thu' : [0,0,0,0,1],
         'Tue' : [0,0,0,1,0],
         'Wed' : [0,1,1,0,0],
         'Autumn' : [1,0,0,0,0],
         'Spring' : [0,0,0,0,0],
         'Summer' : [0,0,0,1,1],
         'Winter' : [0,1,1,0,0],
         'mxpsd' : [12,27,20,8.9,22], 
         'dewp' : [50.3,40.9,21.8,56,57.3], 
         'prcp' : [0,0.33,0.02,0,0],
         'gust' : [0,34,27,0,28],
         'year' : [2017,2017,2015,2014,2015],
         'temp' : [58.8,45,29.9,63,67.3],
         'wdsp' : [7,15.1,12.4,3.5,16.6],
         'sndp' : [0,1.2,2,0,0],
         'fog' : [0,0,0,0,0]
        })

In [ ]:
dnntdf2

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,Fri,Mon,Sat,Sun,Thu,Tue,Wed,Autumn,Spring,Summer,Winter,mxpsd,dewp,prcp,gust,year,temp,wdsp,sndp,fog
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,12.0,50.3,0.00,0,2017,58.8,7.0,0.0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,27.0,40.9,0.33,34,2017,45.0,15.1,1.2,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,20.0,21.8,0.02,27,2015,29.9,12.4,2.0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,8.9,56.0,0.00,0,2014,63.0,3.5,0.0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,22.0,57.3,0.00,28,2015,67.3,16.6,0.0,0


In [ ]:
input = dnntdf1

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_house_regression_trained_model', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
prednorm = format(str(predslistnorm))
print(prednorm)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffbb564e978>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_house_regression_trained_model', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_house_regression_trained_model/mo

In [ ]:
input = dnntdf1.iloc[:,[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 28]]

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_house_regression_trained_model_wstd', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
prednorm = format(str(predslistnorm))
print(prednorm)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb55675ac8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_house_regression_trained_model_wstd', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_house_regression_trained_mod

In [ ]:
input = dnntdf2

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_house_regression_trained_model2', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
prednorm = format(str(predslistnorm))
print(prednorm)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb4d91b860>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_house_regression_trained_model2', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_house_regression_trained_model2/

In [ ]:
input = dnntdf2.iloc[:,[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 28]]

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/DNN_house_regression_trained_model2_wstd', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)

predslistnorm = preds['scores']
prednorm = format(str(predslistnorm))
print(prednorm)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffb5dc2e6a0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/DNN_house_regression_trained_model2_wstd', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/DNN_house_regression_trained_mo

# Conculsions